In [1]:
import numpy as np
import gurobipy as gp
from edge_list import *

In [2]:
cities = list(range(42))
num_cities = len(cities)
distance_matrix = np.zeros((num_cities, num_cities))
for edge in edge_list:
    start_node, end_node, weight = edge
    distance_matrix[start_node][end_node] = weight
    distance_matrix[end_node][start_node] = weight

In [3]:
def ConcatList(o_list, i, j):
    p1_pos = 0
    p2_pos = 0
    for index, sublist in enumerate(o_list):
        if i in sublist:
            p1_pos = index
        if j in sublist:
            p2_pos = index
    if (p1_pos != p2_pos):
        for p in o_list[p2_pos]:
            o_list[p1_pos].append(p)
        del o_list[p2_pos]
    return o_list

In [4]:
# TSP Model
TSP = gp.Model("TSP")
# Vars
arcs = TSP.addVars(cities, cities, vtype='B', name=f'arc')
TSP.update()
# Obj.
TSP.setObjective(
    gp.quicksum(arcs[i, j] * distance_matrix[i][j] for i in cities for j in cities),
    gp.GRB.MINIMIZE
)
# s.t.
for i in cities:
    TSP.addConstr(gp.quicksum(arcs[i, i] for i in cities) == 0)
    TSP.addConstr(gp.quicksum(arcs[i, j] for j in cities) == 1, f'incom_{i}')
    TSP.addConstr(gp.quicksum(arcs[j, i] for j in cities) == 1, f'outgo_{i}')

check_list = []
# When the sub-loop exists
while (len(check_list) != 1):
    # Automatically add constraints
    for sublist in check_list:
        TSP.addConstr(gp.quicksum(arcs[i, j] for i in sublist for j in sublist if i != j) <= len(sublist)-1)
    TSP.setParam('OutputFlag', 0)
    TSP.optimize()
    # Check sub-loop
    check_list = [[i] for i in range(42)]
    if TSP.status == gp.GRB.OPTIMAL:
        for v in TSP.getVars():
            if (v.x!=0):
                i, j = map(int, v.varName[4:-1].split(","))
                check_list = ConcatList(check_list, i, j)
        objValue = TSP.objVal
    else:
        print("No solution found.")
print("Final Cost:", objValue)
for v in TSP.getVars():
    if (v.x!=0):
        print(f"{v.varName}: {v.x}")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-05
Final Cost: 699.0
arc[0,41]: 1.0
arc[1,0]: 1.0
arc[2,1]: 1.0
arc[3,2]: 1.0
arc[4,3]: 1.0
arc[5,4]: 1.0
arc[6,5]: 1.0
arc[7,6]: 1.0
arc[8,7]: 1.0
arc[9,8]: 1.0
arc[10,9]: 1.0
arc[11,10]: 1.0
arc[12,11]: 1.0
arc[13,12]: 1.0
arc[14,13]: 1.0
arc[15,14]: 1.0
arc[16,15]: 1.0
arc[17,16]: 1.0
arc[18,17]: 1.0
arc[19,18]: 1.0
arc[20,19]: 1.0
arc[21,20]: 1.0
arc[22,21]: 1.0
arc[23,22]: 1.0
arc[24,23]: 1.0
arc[25,24]: 1.0
arc[26,25]: 1.0
arc[27,26]: 1.0
arc[28,27]: 1.0
arc[29,28]: 1.0
arc[30,29]: 1.0
arc[31,30]: 1.0
arc[32,31]: 1.0
arc[33,32]: 1.0
arc[34,33]: 1.0
arc[35,34]: 1.0
arc[36,35]: 1.0
arc[37,36]: 1.0
arc[38,37]: 1.0
arc[39,38]: 1.0
arc[40,39]: 1.0
arc[41,40]: 1.0
